<h3><center>Clase ejercicios 02 - IMA539.</center></h3>
<center>Camilo Calfucoy, Jorge Ruíz</center>

<center>Departamento de Ingeniería Matemática</center>

<center>Septiembre 23, 2024.</center>

## Ejercicios propuestos:

1. Cargue la base de datos `Pokemon_DB.csv` en un DataFrame utilizando Pandas.

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score , precision_score ,recall_score

In [ ]:
import pandas as pd
df= pd.read_csv('Pokemon_DB.csv')
df.head()

In [ ]:
print(df.shape)


In [ ]:
df.describe(include='all')

In [ ]:
df.info()

2. Realice un análisis exploratorio de los datos identificando variables numéricas y categóricas. Si las variables categórica no están definidas como tal, modifique su tipo. 

In [ ]:
df.dtypes

In [7]:
df['Name'] = df['Name'].astype('category')
df['Type1'] = df['Type1'].astype('category')
df['Color'] = df['Color'].astype('category')
df['Category'] = df['Category'].astype('category')

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

3. Limpieza de los datos:

    - Cuántas observaciones faltan por cada variable? 
    - Si faltan datos de una variable numérica estos deben rellenarse con el valor promedio de la variable. 
    - Las filas en las cuales falte el dato de alguna variable categórica deben ser eliminadas.
    - Las variables Attack y Speed pero tuvieron problemas al ser ingresadas. Revisar si pueden rectificarse levemente para que puedan ser convertidas a numéricas.

Observaciones:

- La variable Category es categórica y le faltan observaciones.

- HP es numérica y le faltan datos que deben ser completados por valor promedio.

- La Columna Category es categórica y tiene datos faltantes por lo que deben eliminarse esas filas.

- Las Columnas ATTACK y SPEED son realmente variables numéricas, pero contienen errores de ingreso ('j' incluída).
Deben rectificarse algunas entradas para obtener los valores correctos.

In [ ]:
df.isnull().sum()

In [ ]:
print(f'HP es numérica y le faltan {df.HP.isnull().sum()} datos')

In [ ]:
print(f'HP es categótica y le faltan {df.Category.isnull().sum()} datos')

In [13]:
# Eliminando fila con el dato faltante en la variable 'Category'

df_1=df.dropna(subset=['Category'])

In [ ]:
df_1.describe(include='all')

In [ ]:
df_1.isnull().sum()

In [ ]:
HP_promedio=df_1.HP.mean()
HP_promedio

In [ ]:
sum(df_1['HP'].isnull())

In [18]:
df_1.loc[df_1['HP'].isnull(),'HP']=HP_promedio

In [ ]:
df_1.isnull().sum()

In [ ]:
df_1.describe(include='all')

In [ ]:
sum(df_1['HP']==HP_promedio)

In [ ]:
sum(df_1.Attack.apply(lambda x: isinstance(x,str)))

In [ ]:
def noesconvertible_a_float(x):
    try:
        y= float(x)
        return False
    except ValueError:
        return True

df_1.loc[df_1.Attack.apply(noesconvertible_a_float),'Attack']

In [ ]:
def convierte_a_float(x):
    if x.find('j')!=-1:
        x=x.replace('j','')
    return float(x)
    
df_1['Attack']=df_1['Attack'].apply(convierte_a_float)

In [ ]:
df_1.describe()

In [ ]:
sum(df_1.Speed.apply(lambda x: isinstance(x,str)))

In [ ]:
df_1.loc[df_1.Speed.apply(noesconvertible_a_float),'Speed']

In [ ]:
df_1['Speed']=df_1['Speed'].apply(convierte_a_float)

In [ ]:
df_1.describe(include='all')

In [ ]:
df_1.info()

In [ ]:
df_1_describe_numeric= df_1.describe()
df_1_describe_numeric

In [ ]:
# Chequeando Outliers (IQR - Rule)
# lower_bound= Q1 - 1.5*(Q3-Q1)
# upper_bound= Q3 + 1.5*(Q3-Q1)

# Generation
Q1=df_1_describe_numeric.loc['25%']
Q3=df_1_describe_numeric.loc['75%']
IQR=Q3-Q1
print(1.5*IQR)
LB=Q1-1.5*IQR
UB=Q3+1.5*IQR


In [ ]:
UB

In [ ]:
# Lower Bound outliers
Outliers_LB=LB>df_1_describe_numeric.loc['min']
print(Outliers_LB)


In [ ]:
Outliers_LB

In [ ]:

Variables_with_outliers_LB=Outliers_LB.index[Outliers_LB].tolist()
print(Variables_with_outliers_LB)


In [ ]:
type(Variables_with_outliers_LB[0])

outliers_indices=df_1[Variables_with_outliers_LB[0]]< LB[Variables_with_outliers_LB[0]]
rows_with_outliers=set(df_1.index[outliers_indices].tolist())
df_1.loc[outliers_indices,Variables_with_outliers_LB[0]]


In [ ]:

print(f' Variable {Variables_with_outliers_LB[0]} tiene {len(rows_with_outliers)} outliers inferiores.')
#rows_with_outliers
total_outliers_indices=rows_with_outliers

In [ ]:
print(total_outliers_indices)

In [ ]:
# Upper Bound outliers
Outliers_UB=UB<df_1_describe_numeric.loc['max']
print(Outliers_UB)


In [ ]:
Variables_with_outliers_UB=Outliers_UB.index[Outliers_UB].tolist()
print(Variables_with_outliers_UB)


In [ ]:

for variable in Variables_with_outliers_UB:
    outliers_indices=df_1[variable]> UB[variable]
    rows_with_outliers=set(df_1.index[outliers_indices].tolist())
    #df_1.loc[outliers_indices,variable]
    print(f' Variable {variable} tiene {len(rows_with_outliers)} outliers superiores.')
    total_outliers_indices.update(rows_with_outliers)
    
print(f'En total hay {len(total_outliers_indices)} filas con outliers')


In [ ]:
total_outliers_indices

In [47]:
df_1.shape
df_2=df_1.drop(list(total_outliers_indices))

In [ ]:
df_1

In [ ]:
df_2

4. Genere un grafico de tortas que muestre el porcentaje del total de datos
que pertenece a cada clase de la variable `Category` y la cantidad de datos por clase.


In [ ]:
# Solución


5. Reemplazar la columna `Category` por valores numericos 


In [ ]:
# Solución


6. Encuentre dos variables numéricas que no sean linealmente separable y grafíquelas. 

In [ ]:
# Solución


7. Genere una matriz `X` que contenga todas las variables numéricas, que utilizaremos para clasificar. Por otra parte en el vector `y` considere la variable objetivo  `Category`.

In [124]:
# Solución


8. Crear un conjunto de entrenamiento utilizando el 70% de los datos y dejar el restante 30% para el conjunto de testeo.

In [125]:
# Solución


9. Escalar las características de `X` utilizando StandardScaler

In [126]:
# Solución


10. Utilizar el conjunto de entrenamiento para entrenar los siguientes tres modelos de clasificacion:
- SVM
- Random Forest
- KNN
Pruebe con varios hyperparámetros para cada modelo.

In [ ]:
# Solución


11. Prediga sobre el conjunto de testeo con los 3 modelos entrenados.

In [128]:
# Solución


12. Evalúa el rendimiento de los tres modelos usando accuracy como métrica. Indique explique que magnitud refleja esta metrica

In [ ]:
# Solución


13. Escoja el mejor modelo y en genere dos gráficos de torta indicando las proporciones de clases. Un gráfico basado en las clases predichas por el modelo  sobre el conjunto de testeo y otro con las clases reales del test.

In [ ]:
# Solución
